Programare Concurenta si Distribuita
<br>
Tema 1
<br>
Simion Ruben-Andrei MISS1

**How to use the program**
<br>
the application consists of 4 Python scripts:
- the server implementation
- the client implementation
- the experiment pipeline
- a helper script that contains all the settings the experiment pipeline can user and 2 functions use to generate the settings/commandline for the server/client scripts.

The server and the client receive the commands as parameters, as follows:

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--protocol", choices=["tcp", "udp", "quic"])
parser.add_argument("--method", choices=["streaming", "stop-and-wait"])
parser.add_argument("--host")
parser.add_argument("--port", type=int)
parser.add_argument("--termination_signal", description="The termination signal used to close UDP server")
parser.add_argument("--size", type=int, description="The total size of the data")
parser.add_argument("--block_size", type=int, description="Block size, for random use 0")
parser.add_argument("--file_report", description="Where the results will be reported, for console print, ignore argument")
settings = vars(parser.parse_args())

The experiment pipeline will execute the server and client with every possible combination for a number of epochs of experiment, will monitor the execution and close hanging processes in case of error, as follows

In [ ]:
for iteration in range(100):
    for protocol in SETTINGS_TEST_MODES:
        for method in SETTINGS_METHODS:
            for test_size in SETTINGS_TEST_SIZES:
                for block_size in SETTINGS_BLOCK_SIZES:

                    settings = get_settings_json(protocol, method, test_size, block_size)

                    # ... Generating the arguments for both server and client using the settings

                    print("Executing the test:", generate_cmdline(protocol, method, test_size, block_size))

                    server_process = subprocess.Popen(server_args, shell=True)
                    time.sleep(2)
                    client_process = subprocess.Popen(client_args, shell=True)

                    try:
                        # Wait for up to 10 minutes
                        server_process.wait(timeout=600)
                        client_process.wait(timeout=600)
                    except subprocess.TimeoutExpired:
                        print("Test failed for command: ", generate_cmdline(protocol, method, test_size, block_size))
                        server_process.terminate()
                        client_process.terminate()

                        server_process.wait()
                        client_process.wait()

The commandlines can be generated manually using the same function used in the pipeline.

**Statistics and comparisons of performance for each protocol**

The performed tests were the following:

Protocols:
1. TCP
2. UDP
3. QUIC

Modes:
1. Streaming mode
2. Stop-and-wait

Data sizes:
1. 10MB
2. 500MB
3. 1GB

Packets:
1. 1K size packets
2. 32K size packets
3. Random packets ranging from 1 to 64K bytes

The names of the tests in the following tables will follow the following format:
<br>
\<Mode>.\<Size>.\<Packets>
<br>
Example: The test with Streaming Mode, 1Gb size and Random packets will be named 1.3.3


For this comparison, the following tests will be presented:
- Streaming Mode + 500MB + Random packets - 1.2.3
- Stop-and-wait + 1GB + Random packets - 2.3.3
- Streaming mode + 1GB + 32K packets - 1.3.2


Client benchmarks

|     Test    |       Protocol                |     Speed     |    Packets Count | Size sent |
|     :---------:      | :-------:            |     :------:     |    :--------: | :--: |
| 1.2.3 | tcp | 0.5031032562255859 | 16207 | 524289072 |
| 1.2.3 | udp | 0.2928769588470459 | 16133 | 524316795 |
| 1.2.3 | quic | 27.45422863960266 | 16147 | 524302346 |
| 2.3.1 | tcp | 1.0736298561096191 | 33027 | 1073758269 |
| 2.3.1 | udp | 1.5527737140655518 | 32983 | 1073749102 |
| 2.3.1 | quic | 73.76055431365967 | 33082 | 1073753216 |
| 1.3.2 | tcp | 0.9339430332183838 | 32768 | 1073741824 |
| 1.3.2 | udp | 0.6325652599334717 | 32768 | 1073741824 |
| 1.3.2 | quic | 60.24249744415283 | 32768 | 1073741824 |

Server benchmarks

| Test | Protocol | Speed | Packets Received | Size Received | Loss |
| :--: | :------: | :---: | :--------------: | :-----------: | :--: |
| 1.2.3 | tcp | 0.5031032562255859 | 16011 | 524289072 | 0.0 |
| 1.2.3 | udp | 0.29187703132629395 | 16031 | 521087270 | 0.6159491801135175 |
| 1.2.3 | quic | 27.639405250549316 | 11 | 11803 | 99.99774881800738 |
| 2.3.1 | tcp | 1.0726284980773926 | 32355 | 1073758269 | 0.0 |
| 2.3.1 | udp | 1.5522727966308594 | 32983 | 1073749102 | 0.0 |
| 2.3.1 | quic | 74.07993841171265 | 73236 | 85791123 | 92.01016381402857 |
| 1.3.2 | tcp | 0.9339430332183838 | 32228 | 1073741824 | 0.0 |
| 1.3.2 | udp | 0.6325652599334717 | 31207 | 1022590976 | 4.7637939453125 |
| 1.3.2 | quic | 60.00434327125549 | 1 | 1169 | 99.99989112839103 |

**Analysis of the experimental results.**

We can observe in the statistics above that in terms of speed, the UDP is ~50% faster in streaming mode than TCP, but ~50 slower in stop-and-wait mode. This is caused by the ACK message which in cases where the packages are in small sizes, the time can almost double.

We can observe that in terms of stability and package loss, TCP and UDP in wait-and-stop achieve 0% loss, while UDP streaming performs better when packages are in random sizes rather than big fixed packets.

QUIC performed rather poorly, due to the packets not being properly processed by the server, leading to data losses. We can also observe that the packets arrive with a different size, which causes the server to receive many more request than the client send, which causes more stress on the server. This also causes our ACK to be sent differently back, the client receives many more ACKs than messages it sends, which causes the client to never wait for the server. A potential fix for this issue is for the ACK message to include the number of processed bytes, and the client to wait for more ACKs until the processed data matches the data sent.

| Test | Protocol | Speed | Packets Received | Size Received | Loss |
| :--: | :------: | :---: | :--------------: | :-----------: | :--: |
| Random | quic | 2.03314471244812 | 1784 | 2077629 | 80.19352040865452 |
| 1K | quic | 24.274022817611694 | 10241 | 10485760 | 0.0 |
| 32K | quic | 2.105201482772827 | 1813 | 2111418 | 79.86394882202148 |

In the test where the packets where sized below ~1600 which seems to be the split size and in stop-and-wait mode, which removes the server overload, the packets arrive correctly and we have 0% loss.

**Conclusions**

As observed, UDP functions the best in environments where speed matters the most and some loss is perceived as acceptable. Having no ACK reduces stress on the network and allows for more data to be sent/received in short period of time. This is better observed in real-life scenarios where packets are not big, and sending 1GB of data may cause a slowdown due to multiple ACK being sent back.

TCP works best in critical networks where packet order and stability is important. Smaller packets may cause a delay in transmission as observed, where switching from fixed 32K to real life scenarios (between 1 and 64K) generates a slowdown of 15%.

QUIC works best in scenarios where the packets are in smaller sizes. Having bigger QUIC packets causes the server to receive more packets than the client sends, which causes more stress on the server. QUIC works best in stream mode, where the client sends 1GB of information in a stream that is processed and merged back on the server. QUIC does not seem suitable in scenarios where packet integrity is important(as we observed, packets are split on the server).